# Some Notes for this notebook

## graph-tool installation

In [2]:
#on windows: 
#use the windows key
#type anaconda
#open the anaconda power prompt as ADMINISTRATOR (right click). 
#conda activate base
#conda create -n myenv python=3.8. 
#conda install -c conda-forge graph-tool

# Functions

## Basic file process

In [1]:
# check your python installation
import sys
sys.executable

'/Users/yil1/opt/anaconda3/envs/P2P/bin/python'

In [2]:
def install_missing_packages(package_names):
    """
    Install Missing Packages

    This function checks if a list of packages is already installed and installs any missing packages using pip.

    Parameters:
    - package_names (list): A list of package names to be installed.

    Returns:
    - None

    Note: This function requires the `subprocess` and `importlib` modules to be imported.

    Example Usage:
    install_missing_packages(['h2o', 'numpy', 'pandas'])
    """
    import importlib
    import subprocess


    for package_name in package_names:
        try:
            importlib.import_module(package_name)
            print(f"{package_name} package is already installed")
        except ImportError:
            print(f"{package_name} package not found, installing with pip...")
            subprocess.call(['pip', 'install', package_name])


In [3]:
package_list = ["networkx", "gower","statsmodels","pyarrow","seaborn", "lime", "shap"]
install_missing_packages(package_list)

networkx package is already installed
gower package is already installed
statsmodels package is already installed
pyarrow package is already installed
seaborn package is already installed
lime package is already installed


/Users/yil1/opt/anaconda3/envs/P2P/lib/python3.8/site-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
/Users/yil1/opt/anaconda3/envs/P2P/lib/python3.8/site-packages/shap/utils/_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def delta_minimiz

shap package is already installed


In [4]:
'''
clean_data(df):
This code defines a function called `clean_data`, which takes a DataFrame (usually a DataFrame object in the pandas library) as input and performs a series of data cleaning operations on it. The specific cleaning steps are as follows:

1. **Delete missing values**: Use the `dropna()` method to remove rows containing missing values.

2. **Filter specific rows**: keep the rows whose `lang.1` column is equal to 1, and reset the index.

3. **Delete 'lang' column**: Delete all columns containing "lang".

4. **Delete Date Columns**: Delete `date.start` and `date.end` columns as they are considered irrelevant.

5. **Remove Look-Bias Variables**: Define a list containing look-ahead-bias variables and use the `drop()` method to drop those columns from the DataFrame.

6. **Delete Duplicate Income Variables**: Delete all columns that contain "inc." and do not contain ".no".

7. **Delete Dummy Variables**: Defines a list of dummy variables to delete and removes these columns from the DataFrame.

8. **Generate Correlation Matrix**: Use the `corr()` method to generate the correlation matrix of DataFrame.

9. **Choose the upper triangle of the correlation matrix**: To avoid multicollinearity, select the upper triangle of the correlation matrix.

10. **Find High Correlation Columns**: Find columns that have a correlation higher than 0.95 with other columns.

11. **Delete High Correlation Columns**: Delete the high correlation columns found in step 10.

Finally, the function returns the cleaned DataFrame.

The purpose of this code is to prepare the data for subsequent analysis or modeling. It cleans data by removing missing values, filtering specific rows, removing irrelevant or biased columns, and handling multicollinearity.

'''


# Import necessary libraries
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
import random

# Define a function to calculate variance inflation factor (VIF) for all variables in a given DataFrame.
def calculate_vif(df):
    """Calculates variance inflation factor for all columns in df. It should contain 
    only exogeneous variables."""
    
    # Create an empty DataFrame
    vif = pd.DataFrame()
    
    # Calculate VIF for every column (variable) in the DataFrame
    vif["VIF Factor"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    
    # Store column names
    vif["features"] = df.columns
    
    # Sort by VIF Factor in descending order
    vif = vif.sort_values("VIF Factor", ascending=False)
    return vif

# Define a function to clean the DataFrame, removing irrelevant and problematic features.
def clean_data(df):
    """
    Clean the given DataFrame, drop unnecessary columns, handle missing data, remove biased variables, 
    avoid multicollinearity by checking correlation, and keep only relevant columns.
    """
    
    # Drop rows with missing values
    df = df.dropna().reset_index(drop=True)

    # Filter for only rows where 'lang.1' equals 1, and drop 'lang' columns
    df = df[df["lang.1"] == 1].reset_index(drop=True)
    df = df.drop([x for x in df.columns if "lang" in x], axis=1)

    # Drop date columns as they are not considered relevant
    df = df.drop(["date.start", "date.end"], axis=1)

    # List of forward-looking biased variables
    fwl_bias = [
        "return",
        "RR1",
        "RR2.Mean",
        "RR2.Median",
        "RR2.WMean",
        "NPRP",
        "NPRA",
        "FVCI",
        "FVCI.Mean",
        "FVCI.Median",
        "FVCI.WMean",
    ]
    # Drop these forward-looking biased variables
    df = df.drop(fwl_bias, axis=1)

    # Drop duplicate income variables, keep only those with '.no' in name
    df = df.drop([x for x in df.columns if "inc." in x and ".no" in x], axis=1)

    # List of dummy variables to drop
    dummies_to_drop = ["AA", "educ.6", "em.dur.5p", "use.m", "ver.2", "Mining", "Utilities"]
    df = df.drop(dummies_to_drop, axis=1)

    # Generate a correlation matrix of the DataFrame
    corr_matrix = df.corr().abs()
    
    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    
    # Find columns with correlation greater than 0.95
    to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
    
    # Drop these columns from the DataFrame
    df = df.drop(df[to_drop], axis=1)

    return df

def create_balanced_sample(df, n, replace = False):
    """Create a balanced sample of size 2n from df. Ensures that the sample contains an equal number of instances for each class."""
    
    # Draw a random sample of size 'n' from the non-default class (default = 0) and from the default class (default = 1)
    # replace=True allows for resampling
    # Random state ensures reproducibility
    df_sample = pd.concat(
        [
            df[df["default"] == 0].sample(n=n, random_state=1, replace=replace),
            df[df["default"] == 1].sample(n=n, random_state=1, replace=replace),
        ]
    ).reset_index(drop=True)
    
    return df_sample


In [5]:
import os

def generate_file_path(file_name):
    """
    Generate a file path for a file located one directory level up.

    Parameters:
    file_name (str): The name of the file including any subdirectories from the parent directory.

    Returns:
    file_path (str): The full path to the file.
    """
    # Get the current working directory
    cwd = os.getcwd()

    # Get the parent directory
    parent_dir = os.path.dirname(cwd)

    # Define the file path by joining the parent directory path with the file name
    file_path = os.path.join(parent_dir, file_name)

    return file_path


In [6]:
# function to save a pandas dataframe as csv to disc.
def save_df_to_csv(df, file_path):
    """
    This function saves a pandas DataFrame to a CSV file.

    Parameters:
    df (pandas.DataFrame): The DataFrame to save.
    file_path (str): The file path where to save the DataFrame, including the filename.

    Returns:
    None
    """
    df.to_csv(file_path, index=False)  # Set index=False to not save row indices

In [7]:
import pandas as pd

def compute_summary_stats(df, cols):
    """
    This function computes summary statistics for specified columns in a pandas dataframe.

    Args:
    df (pandas.DataFrame): The dataframe on which to compute summary statistics.
    cols (list): A list of column names for which to compute summary statistics.

    Returns:
    pandas.DataFrame: A dataframe with summary statistics for the specified columns.

    Example:
    summary_stats = compute_summary_stats(df, ['liab.I', 'inc.total', 'MonthlyPayment', 'log.amount', 'time', 'Interest', 'AmountOfPreviousLoansBeforeLoan', 'NoOfPreviousLoansBeforeLoan', 'Age'])
    print(summary_stats)
    """

    # Check if all columns exist in dataframe
    for col in cols:
        if col not in df.columns:
            print(f'Column {col} does not exist in the dataframe.')
            return None

    # Compute summary statistics
    summary_stats = df[cols].describe()

    return summary_stats

In [8]:
def df_to_latex(df):
    """
    This function converts a pandas DataFrame into a LaTeX table.

    Args:
    df (pandas.DataFrame): The dataframe to convert to LaTeX.

    Returns:
    str: A string of LaTeX code for a table with the data from the DataFrame.

    Example:
    latex_code = df_to_latex(summary_stats)
    print(latex_code)
    """

    # Convert DataFrame to LaTeX
    latex_code = df.to_latex()

    return latex_code

In [9]:
# retrieving the number of non-defaulted vs. defaulted loans
def count_defaults(df):
    """
    This function counts and prints the number of defaulted loans in a DataFrame.
    
    Parameters:
    df (DataFrame): The DataFrame containing the loan data. It must contain a 'default' column 
                    with binary values: 1 for default and 0 otherwise.

    Returns:
    None. The function directly prints the number of defaulted loans.
    """
    
    num_defaults = df['default'].sum()
    print(f"The number of loans that have defaulted is: {num_defaults}")

## Gowers distance


In [10]:
import pandas as pd
import numpy as np
import gower


def calculate_and_save_gowers_distance(dataframe, output_file_path):
    """
    Calculates Gower's distance matrix for a given DataFrame and saves the result as a numpy array.

    Parameters:
    dataframe (pandas.DataFrame): The input DataFrame.
    output_file_path (str): The path where the resulting numpy array should be saved.

    Returns:
    None
    """
    try:
        # Drop the 'default' column to keep only features
        dataframe = dataframe.drop(["default"], axis=1)

        # Identify dummy columns (faster for gower calculations)
        dummy_columns = [
            column for column in dataframe.columns
            if ((dataframe[column] == 0) | (dataframe[column] == 1)).all()
        ]
        categorical_variables = [
            column in dummy_columns for column in dataframe.columns
        ]

        # Calculate Gower's distance
        distance_matrix = gower.gower_matrix(
            dataframe, cat_features=categorical_variables)

        # Save the distance matrix as a numpy array
        np.save(output_file_path, distance_matrix)

        print(f"Distance matrix saved successfully at {output_file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")
    return distance_matrix


# Changes made:

# 1. Renamed the function to `calculate_and_save_gowers_distance` for clarity.
# 2. Added a try-except block to handle potential errors during execution.
# 3. Added comments to explain each step of the process.
# 4. Improved the print statement to include the output file path for better tracking.

In [11]:
import numpy as np

def load_gowers_distance_matrix(file_path):
    """
    Loads Gower's distance matrix from a given file path.

    Parameters:
    file_path (str): The path from where the numpy array should be loaded.

    Returns:
    numpy.ndarray: Loaded Gower's distance matrix.
    """
    try:
        distance_matrix = np.load(file_path)
        print(f"Distance matrix loaded successfully from {file_path}")
        return distance_matrix
    except Exception as e:
        print(f"An error occurred: {e}")


In [12]:
import numpy as np
import pandas as pd
from graph_tool.all import *

In [ ]:
def matrix_to_graph_tool(adj):
    """
    Convert adjacency matrix to graph using graph-tool.

    Parameters:
    adj (numpy.ndarray): The adjacency matrix.

    Returns:
    g (graph_tool.Graph): The graph.
    """
    # Extract index and weights from the adjacency matrix.
    # np.nonzero(np.triu(adj, 1)) returns the indices of the upper triangle of the matrix,
    # excluding the diagonal. This is because for an undirected graph, the adjacency matrix
    # is symmetric, and we only need to consider half of the matrix to get all the edges.
    idx = np.nonzero(np.triu(adj, 1))

    # Get the weights of the edges from the adjacency matrix.
    weights = adj[idx]

    # Create an empty graph.
    g = Graph()

    # Add edges to the graph. np.transpose(idx) gives a 2D array where each row is the indices
    # of the two vertices of an edge.
    g.add_edge_list(np.transpose(idx))

    # Create an edge property map for the weights of the edges.
    edge_weight = g.new_edge_property("double")

    # Assign the weights to the edge property map.
    edge_weight.a = weights

    # Add the edge property map to the graph.
    g.edge_properties["edge_weight"] = edge_weight

    return g

def calculate_centrality_measures(g):
    """
    Calculate various centrality measures for the graph.

    Parameters:
    g (graph_tool.Graph): The graph.

    Returns:
    df (pandas.DataFrame): DataFrame with centrality measures.
    """
    # Calculate PageRank centrality. This measure reflects the importance of a node in the graph.
    # Nodes with a high PageRank centrality are those that are well connected or connected to well-connected nodes.
    pr_w = pagerank(g, weight=g.ep.edge_weight).a

    # Calculate betweenness centrality. This measure reflects the amount of control that a node exerts over the interactions of other nodes in the graph.
    # Nodes with high betweenness centrality are those that lie on many shortest paths between other nodes.
    bw_w = betweenness(g, weight=g.ep.edge_weight)[0].a

    # Calculate closeness centrality. This measure reflects how close a node is to all other nodes in the graph.
    # Nodes with high closeness centrality can reach other nodes quickly.
    cl_w = closeness(g, weight=g.ep.edge_weight).a

    # Calculate eigenvector centrality. This measure reflects the influence of a node in a network.
    # Nodes with high eigenvector centrality are those connected to many nodes who themselves have high centrality.
    ev_w = eigenvector(g, weight=g.ep.edge_weight)[1].a

    # Calculate Katz centrality. This measure is a generalization of degree centrality and eigenvector centrality.
    kt_w = katz(g, weight=g.ep.edge_weight).a

    # Calculate HITS authority and hub scores. The HITS algorithm computes two numbers for a node: 
    # Authorities estimates the node value based on the incoming links. 
    # Hubs estimates the node value based on outgoing links.
    hits_aut_w = hits(g, weight=g.ep.edge_weight)[1].a
    hits_hub_w = hits(g, weight=g.ep.edge_weight)[2].a

    # Create DataFrame
    df = pd.DataFrame({
        "pagerank": pr_w,
        "betweenness": bw_w,
        "closeness": cl_w,
        "eigenvector": ev_w,
        "katz": kt_w,
        "authority": hits_aut_w,
        "hub": hits_hub_w,
    })

    return df


## Plot

In [28]:
import numpy as np
import matplotlib.pyplot as plt
import os


def save_histogram_pdf(array, file_path, samplesPerBin):
    """
    Saves a histogram of a numpy array to a given file path in PDF format.

    Parameters:
    array (numpy.ndarray): The input numpy array.
    file_path (str): The path where the PDF should be saved.
    samplesPerBin (int): The desired number of samples per bin.

    Returns:
    None
    """
    try:
        # Calculate the number of bins
        num_bins = array.size // samplesPerBin
        if array.size % samplesPerBin > 0:  # if there's a remainder, add an extra bin
            num_bins += 1

        # Ensure we have at least 1 bin
        num_bins = max(1, num_bins)

        # Create directory if it does not exist
        directory = os.path.dirname(file_path)
        if not os.path.exists(directory):
            os.makedirs(directory)

        plt.hist(array.flatten(), bins=num_bins)
        plt.title('Histogram of ndarray values')
        plt.xlabel('Value')
        plt.ylabel('Frequency')
        plt.savefig(file_path, format='pdf')
        plt.close()
        print(f"Histogram saved successfully at {file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")


# Read back distance matrix

In [14]:
num_default_samples=12000

In [15]:
dist_matrix_path = generate_file_path("data//Dist_matrix("+str(num_default_samples*2)+").npy")
print(dist_matrix_path)

/Users/yil1/p2p-model-bondora/data//Dist_matrix(24000).npy


In [16]:
distance_matrix=load_gowers_distance_matrix(dist_matrix_path)

Distance matrix loaded successfully from /Users/yil1/p2p-model-bondora/data//Dist_matrix(24000).npy


In [17]:
print(type(distance_matrix))
print(distance_matrix.shape)

<class 'numpy.ndarray'>
(24000, 24000)


In [18]:
dist_matrix_plot_path = generate_file_path("graph/paper2/Dist_matrix("+str(num_default_samples*2)+").pdf")
print(dist_matrix_plot_path)

/Users/yil1/p2p-model-bondora/graph/paper2/Dist_matrix(24000).pdf


In [31]:
save_histogram_pdf(array=distance_matrix,
                   file_path=dist_matrix_plot_path,
                   samplesPerBin=10000)

Histogram saved successfully at /Users/yil1/p2p-model-bondora/graph/paper2/Dist_matrix(24000).pdf


# Add centrality measures
more on graph-tools.
https://robert-haas.github.io/gravis-docs/code/examples/external_tools/graph-tool.html

## Add centrality measures
Wrapped the graph creation and centrality measure calculation into separate functions for better readability and reusability.
Removed unnecessary list ls. Instead, created the DataFrame directly using a dictionary.
Added docstrings to the functions to explain what they do.
Removed the unused time import.
Added comments to explain each step of the process.
Used pd.concat() to merge the original DataFrame with the centrality measures DataFrame. This is more efficient and readable than creating a new DataFrame and then adding columns to it.

In a weighted graph, the weight of an edge can represent various things depending on the context of the problem. It could represent the strength of the connection between the nodes, the distance between the nodes, the cost of moving from one node to another, etc.

The way the weight of an edge affects the centrality measures depends on the specific measure:

1. **PageRank**: In the context of PageRank, the weight of an edge can be interpreted as the probability of a random walk following the edge. A higher weight means that it's more likely for the random walk to follow the edge. Therefore, nodes connected by high-weight edges will have a higher PageRank centrality.

2. **Betweenness**: In the context of betweenness centrality, the weight of an edge can be interpreted as the cost of traversing the edge. A lower weight means that it's less costly to traverse the edge. Therefore, nodes connected by low-weight edges will have a higher betweenness centrality.

3. **Closeness**: In the context of closeness centrality, the weight of an edge can be interpreted as the distance between the nodes. A lower weight means that the nodes are closer to each other. Therefore, nodes connected by low-weight edges will have a higher closeness centrality.

4. **Eigenvector**: In the context of eigenvector centrality, the weight of an edge contributes to the centrality of the nodes it connects. A higher weight means that the nodes will have a higher eigenvector centrality.

5. **Katz**: In the context of Katz centrality, the weight of an edge contributes to the centrality of the nodes it connects. A higher weight means that the nodes will have a higher Katz centrality.

6. **HITS**: In the context of HITS, the weight of an edge contributes to the authority and hub scores of the nodes it connects. A higher weight means that the nodes will have a higher authority and hub scores.

In all these measures, the weight of an edge is used to quantify the importance of the edge in the graph. The exact interpretation of the weight depends on the context of the problem and the specific centrality measure.

## Understand adjacency and distance matrix
A Gower's distance matrix and an adjacency matrix are both square matrices that represent relationships between entities, but they are used in different contexts and convey different types of information.

1. **Gower's Distance Matrix**: This matrix is used in the context of cluster analysis or hierarchical clustering. Each entry in the matrix represents the Gower's distance between two entities (e.g., individuals, observations, etc.). Gower's distance is a measure of dissimilarity between two entities and takes into account both numerical and categorical variables. The smaller the Gower's distance, the more similar the two entities are.

2. **Adjacency Matrix**: This matrix is used in the context of graph theory. Each entry in the matrix indicates whether two vertices (i.e., nodes) in a graph are adjacent to each other. In an unweighted graph, the entries are binary (0 or 1), indicating the absence or presence of an edge between two vertices. In a weighted graph, the entries can be any non-negative real number, representing the weight of the edge between two vertices.

In some cases, a Gower's distance matrix could be transformed into a form of adjacency matrix. For example, you might create a graph where each entity is a node, and an edge exists between two nodes if their Gower's distance is below a certain threshold. The resulting adjacency matrix would represent a graph of entities that are similar to each other according to the Gower's distance. However, this would be a specific use case and not a general relationship between Gower's distance matrices and adjacency matrices.

## Explanation
1. **Adjacency Matrix**: An adjacency matrix is a square matrix used to represent a finite graph. In an unweighted graph, the elements of the adjacency matrix (A[i][j]) indicate whether pairs of vertices are adjacent or not in the graph -- 1 for adjacent, 0 for not adjacent. In a weighted graph, the elements of the adjacency matrix represent the weights of the edges.

2. **Distance Matrix**: A distance matrix is a square matrix containing the distances, taken pairwise, between the elements of a set. In an unweighted graph, the elements of the distance matrix (D[i][j]) represent the number of edges in the shortest path between each pair of vertices. In a weighted graph, the elements of the distance matrix represent the sum of the weights of the shortest path between each pair of vertices.

3. **Gower's Distance Matrix**: Gower's distance matrix is a special type of distance matrix primarily used for mixed data types. It is not commonly used in graph theory.

In [51]:
# Load adjacency matrix
# The adjacency matrix is a square matrix used to represent a finite graph.
# The elements of the matrix indicate whether pairs of vertices are adjacent or not in the graph.
adj_path = generate_file_path("data//Dist_matrix("+str(num_default_samples*2)+").npy")
adj = np.load(adj_path)

Explain: Here, this graph is a fully connected, which means each pair of nodes has an edge. Thus, the Adjacency Matrix is equivalent to the Gower's Distance Matrix.

(When a graph is not fully connected, these two matrix are not equivalent any more.)

In [52]:
# Convert adjacency matrix to graph
# The adjacency matrix is then converted into a graph object using the `matrix_to_graph_tool` function.
# This function takes the adjacency matrix as input and returns a graph object that can be manipulated using the `graph-tool` library.
g = matrix_to_graph_tool(adj)

Attention here! We can use alpha threathold here!

In [53]:
# Calculate minimum spanning tree and set as edge filter
# A minimum spanning tree (MST) of a graph is a subset of the edges of the graph that connects all the vertices together,
# without any cycles and with the minimum possible total edge weight.
# The `min_spanning_tree` function from the `graph-tool` library is used to calculate the MST.
# The resulting MST is then set as an edge filter on the graph using `g.set_edge_filter(tree)`.
# This means that only the edges that are part of the MST will be considered in subsequent operations on the graph.
tree = min_spanning_tree(g, weights=g.ep.edge_weight)
g.set_edge_filter(tree)

In [54]:
g

<Graph object, directed, with 24000 vertices and 23999 edges, 1 internal edge property, edges filtered by (<EdgePropertyMap object with value type 'bool', for Graph 0x2ab455e80, at 0x17ff403a0>, False), vertices filtered by (<VertexPropertyMap object with value type 'bool', for Graph 0x2ab455e80, at 0x17ab72c10>, False), at 0x2ab455e80>

In [55]:
# Calculate centrality measures
# Finally, several centrality measures are calculated for the graph using the `calculate_centrality_measures` function.
# Centrality measures provide a way of identifying the most important vertices within a graph.
# They are commonly used in network analysis to determine the relative importance of a vertex within the graph.
# The resulting centrality measures are stored in a DataFrame.
centrality_df = calculate_centrality_measures(g)

In [56]:
centrality_df

,pagerank,betweenness,closeness,eigenvector,katz,authority,hub
0,0.000023,0.0,8.902073,NaN,0.006453,0.000000e+00,2.078653e-22
1,0.000023,0.0,15.976149,NaN,0.006453,0.000000e+00,2.410830e-83
2,0.000023,0.0,13.396533,NaN,0.006453,0.000000e+00,6.701703e-47
3,0.000023,0.0,7.359868,NaN,0.006453,0.000000e+00,3.883548e-67
4,0.000023,0.0,26.029100,NaN,0.006453,0.000000e+00,1.762067e-95
...,...,...,...,...,...,...,...
23995,0.000127,0.0,NaN,NaN,0.006456,6.289251e-103,0.000000e+00
23996,0.000078,0.0,NaN,NaN,0.006454,3.161870e-142,0.000000e+00
23997,0.000184,0.0,NaN,NaN,0.006455,5.968064e-126,0.000000e+00
23998,0.000034,0.0,NaN,NaN,0.006454,4.234108e-49,0.000000e+00


In [57]:
type(centrality_df)

pandas.core.frame.DataFrame

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages

def plot_histogram(df, column):
    """
    对于每列，绘制直方图。
    如果这一列全是NaN，输出警告并跳过。
    """
    if df[column].isna().all(): # 如果列全是NaN值，输出警告并跳过
        print(f"Warning: Column {column} only contains NaN values. Skipping...")
        return

    plt.figure()
    sns.histplot(df[column], kde=False, bins=50) # 限制bins数量为50
    plt.title(f"Histogram of {column}")
    plt.xlabel(column)
    plt.ylabel('Frequency')

centrality_data_path = generate_file_path("graph/Bondora_centrality_sample("+str(num_default_samples*2)+").pdf")

with PdfPages(centrality_data_path) as pdf: # 输出至PDF
    for column in centrality_df.columns:
        plot_histogram(centrality_df, column)
        if plt.get_fignums():  # 检查是否有活动的figure
            pdf.savefig()  # 将当前figure保存到pdf中
            plt.close()    # 关闭当前figure


## Combine ans save files

In [58]:
# Load original dataset
sample_data_path = generate_file_path("data/Bondora_sample("+str(num_default_samples*2)+").feather")
df_sample = pd.read_feather(sample_data_path)
print(sample_data_path)

# Merge with original dataset
df_sample_with_centrality = pd.concat([df_sample, centrality_df], axis=1)

/Users/yil1/p2p-model-bondora/data/Bondora_sample(24000).feather


In [59]:
df = df_sample_with_centrality

# Define the text file path
txt_file_path = generate_file_path("data//Bondora_sample("+str(num_default_samples*2)+")_with_centrality.csv")
print(txt_file_path)
# Write the DataFrame to the text file
df.to_csv(txt_file_path, sep=',', index=False)

/Users/yil1/p2p-model-bondora/data//Bondora_sample(24000)_with_centrality.csv
